In [1]:
import time
start_time = time.time()

import numpy as np
from keras.models import load_model
import os
import scipy.io


def load_scaling_factors(file_path):
    return np.loadtxt(file_path, delimiter=',', skiprows=1)

def generate_ranges(ranges):
    return np.unique(np.concatenate([np.arange(start, stop + step, step) for start, stop, step in ranges]))

def normalize(data, min_val, max_val):
    return (data - min_val) / (max_val - min_val)

def denormalize(data, min_val, max_val):
    return data * (max_val - min_val) + min_val


# File paths and model loading
path = "/Users/janreifferscheidt/Documents/Training/"
model = load_model("GRU_Full_01-28_LR_1E-03_BS_8_att_pp")


# RR_Intervals and ranges
RR_Intervals = [
    1010.30000000261, 1005.24999999441, 989.800000004470, 990.149999996647,
    1020.10000000708, 984.899999992922, 999.800000004470, 1005.04999999888,
    1005.14999999665, 1004.90000000224, 1035.09999999776, 1030,
    989.900000002235, 989.599999999628
]
t1_ranges = [(50, 500, 50), (500, 1000, 5), (1000, 1500, 50), (1500, 2000, 100)]
t2_ranges = [(4, 80, 1), (80, 200, 5), (200, 400, 20)]



# Load scaling factors
params_file_path = os.path.join(path, 'train', 'Scaling_Factors_Params.txt')
finger_file_path = os.path.join(path, 'train', 'Scaling_Factors_Finger.txt')

scaling_factors_finger = load_scaling_factors(finger_file_path)
realMin, realMax, imagMin, imagMax = scaling_factors_finger
scaling_factors_params = load_scaling_factors(params_file_path)
min_vals_t, max_vals_t, min_vals_rr, max_vals_rr = scaling_factors_params


# Generate T1 and T2 arrays and combinations
T1 = generate_ranges(t1_ranges)
T2 = generate_ranges(t2_ranges)
combinations_matrix = np.array([[t1, t2] for t2 in T2 for t1 in T1]) #if t1 >= t2

# Prepare ParamsMatrix
ParamsMatrix = np.hstack([combinations_matrix, np.zeros((combinations_matrix.shape[0], 14))])
ParamsMatrix[:, 2:] = RR_Intervals

# Normalize for GRU
t_times = ParamsMatrix[:, 0:2]
rr_times = ParamsMatrix[:, 2:]
scaled_data_t = normalize(t_times, min_vals_t, max_vals_t)
scaled_data_rr = normalize(rr_times, min_vals_rr, max_vals_rr)
normalized_ParamsMatrix = np.hstack((scaled_data_t, scaled_data_rr))

# Model prediction
Dic_gen = model.predict(normalized_ParamsMatrix)

# Denormalize
real_part = denormalize(Dic_gen[:, :Dic_gen.shape[1] // 2], realMin, realMax).astype(np.float64)
imaginary_part = denormalize(Dic_gen[:, Dic_gen.shape[1] // 2:], imagMin, imagMax).astype(np.float64)
Dic = real_part + 1j * imaginary_part

#turning Unphysical parameter to 0
indices = np.where(ParamsMatrix[:, 1] > ParamsMatrix[:, 0])[0]
Dic[indices, :] = 0 + 0j

# Save to .mat file
matfile_data = {'GenDict': {'dictOn': Dic, 'Combinations': ParamsMatrix}}

scipy.io.savemat('GenDict_NN.mat', matfile_data)

# Calculate execution time
execution_time = time.time() - start_time
print(f"Execution Time: {execution_time} seconds")



434/434 [==============================] - 3s 6ms/step
Execution Time: 8.40018105506897 seconds
